In [1]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}   
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, GlobalAveragePooling1D, DepthwiseConv2D, MaxPooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Add
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

IMAGE_SIZE = 96

def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32):

     # dataset parameters
     train_path = os.path.join(base_dir,'train+val','train')
     valid_path = os.path.join(base_dir,'train+val','valid')


     RESCALING_FACTOR = 1./255

     # instantiate data generators
     datagen = ImageDataGenerator(rescale=RESCALING_FACTOR)

     train_gen = datagen.flow_from_directory(train_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode='binary')

     val_gen = datagen.flow_from_directory(valid_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode='binary')

     return train_gen, val_gen

os.chdir('..')
dir=os.getcwd()


c:\Users\20222787\.conda\envs\tf\lib\site-packages\tensorflow\python\keras\engine\training_arrays_v1.py:37: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.5)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top


In [3]:
# et the filepath where the trained model is saved (should be in the 'metadata' directory with a .keras extension)
model_name='CoAtNet_2conv_4pool_2ViTdrp.3_drp.3_10e_btc32'
model_filepath = dir+'/metadata/'+model_name + '_weights.keras'

# load the model from the given filepath
model=tf.keras.models.load_model(model_filepath)

train_gen, val_gen = get_pcam_generators(dir+'\Data')

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [4]:
y_pred_prob = list(model.predict(val_gen).flatten())  # Returns a probability per image
y_pred=[1 if num >= .5 else 0 for num in y_pred_prob]
y_true = list(val_gen.classes)  # The real labels (0 or 1) from the validation set

accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='binary') 

print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')

500/500 [==============================] - 176s 336ms/step
Accuracy: 0.4930
Recall: 0.5683


In [4]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    
# Compute the AUC
roc_auc = auc(fpr, tpr)
print(roc_auc)

0.498670640625


In [6]:
from sklearn.metrics import confusion_matrix as cm
cm(y_pred,y_true)

array([[3947, 3983],
       [4053, 4017]], dtype=int64)